# Django ORM - Instrukcje DML w relacjach

W bazach relacyjnych tabele mogą być powiązane ze sobą relacjami. W zależności od natury tego powiązania wyróżniamy kilka typów relacji. W Django ORM możemy wyróżnić trzy podstawowe typy relacji: jeden-do-jednego, jeden-do-wielu i wiele-do-wielu. Każda z tych relacji posiada w Django ORM swój odpowiednik w postaci pola modelu. I tak:
* **OneToOneField** to pole odpowiadające relacji jeden do jednego
* **ForeignKey** to pole odpowiadające relacji jeden do wielu
* **ManyToManyField** to pole odpowiadające relacji wiele do wielu

Omówmy je po kolei.

## OneToOneField

Relacja jeden do jednego występuje w przypadku kiedy rekord jednej tabeli może być powiązany z jednym i tylko jednym wpisem drugiej tabeli. Przykładem takiej relacji może być tabela stolica oraz tabela państwo. Warszawa jest stolicą tylko jednego państwa - Polski, Polska ma przypisaną tylko jedną stolicę - Warszawę.
"Mówimy Warszawa myślimy Polska, mówimy Polska myślimy Warszawa". Relacja jest symetryczna dlatego nie ma znaczenia, w której z tabelek (stolica, czy państwo) umieścimy kolumnę dla tej relacji.

**Definicje modeli**

W modelach mamy dwie klasy: Country i Capitol. Relacje OneToOneField umieściliśmy po stronie modelu Country. Atrybut przechowujący relację nazwaliśmy capitol.

<code>class Country(models.Model):
    name = models.CharField(max_length=64)
    capitol = models.OneToOneField('Capitol', on_delete=models.CASCADE)
</code>

<code>class Capitol(models.Model):
    name = models.CharField(max_length=64)
</code>

### C z CRUD

In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from relation_app.models import Country, Capital

In [3]:
# 1. Tworzymy wpis w tabeli Capital
warsaw = Capital.objects.create(name="Warsaw")

In [6]:
# 2. Tworzymy wpis tabeli Country
poland = Country.objects.create(name="Poland")

IntegrityError: NOT NULL constraint failed: relation_app_country.capital_id

In [ ]:
# Nie możemy! Dostaliśmy IntegrityError.
# Dlaczego?
# Ponieważ pole capitol (z referencją do modelu Capital) nie może być puste, a my próbując stworzyć 
# wpis nie podaliśmy wartości w tym polu.

# Spróbujmy dopisać
poland = Country.objects.create(name="Poland", capital="Warsaw")

In [ ]:
# Znów się nie udało. Tym razem wprowadziliśmy wartość, ale była ona typu string. 
# Python informuje nas, że wprowadzana w tym polu wartość powinna być instancją klasy 
# (modelu) Capital.

# Ale my już mamy taką jedną instancję. Przypisaliśmy ją do zmiennej warsaw, tworząc 
# pierwszy wpis w tabeli Captial.
warsaw

In [ ]:
# czyli wystarczy teraz ją wykorzystać. To pierwszy sposób tworzenia.

# Metada I (instancja modelu)
poland = Country.objects.create(name="Poland", capital=warsaw)

In [ ]:
# Oczywiście możemy używać tu również dwóch pozostałych metod tworzenia wpisów, 
# które już poznaliśmy.

# Tworzymy wpis dla Francji (metoda II)
paris = Capital(name="Paris")
paris.save()
france = Country(name="France", capital=paris)
france.save()

# Tworzymy wpis dla Włoch (metoda III)
rome = Capital()
rome.name = "Rome"
rome.save()
italy = Country()
italy.name = "Italy"
italy.capital = rome
italy.save()

In [ ]:
# Ale mamy jeszcze inny sposób. 

# Popatrzmy na to co posiada tabelka Country
print(dir(Country))

In [ ]:
# Widać, że Django poza polem capitol wygenerowało również dla modelu Country pole capitol_id 
# reprezentujące id wpisu tabeli Capitol. Możemy użyć tego pola do utworzenia relacji.

# Metoda II (id instancji modelu)

# 1. Tworzymy wpis w tabeli Capitol
berlin = Capital.objects.create(name="Berlin")
berlin_id = berlin.id  # -> 2

# 2. Tworzymy wpis w tabeli Country przekazując id instancji modelu Capitol
germany = Country.objects.create(name="Germany", capital_id=berlin_id)

## ForeignKey

Relacja jeden do wielu jest najczęściej wykorzystywanym typem relacji. Występuje wtedy, kiedy wpis z jednej tabeli (tzw. tabeli rodzica) może być powiązany z wieloma wpisami z drugiej (tzw. tabeli dziecka), ale wpis z drugiej tabeli (tabeli dziecka) nie może być powiązany z wielom wpisami z pierwszej (tabeli rodzica). Innymi słowy rodzic może mieć wiele dzieci, ale dziecko może mieć tylko jednego rodzica. Przykładem takiej relacji może być tabela "miasto" oraz tabela "państwo". Gdańsk należy do Polski. Kraków należy do Polski. Mówiąc Gdańsk myślimy Polska (to strona relacja "jeden"). Podobnie mówiąc Kraków myślimy Polska. Ale już mówiąc Polska myślimy Gdańsk, Kraków, Wrocław, ... (to strona relacji "wiele"). 

Najprościej jest wyobrazić sobie relacje jeden-do-wielu jako strukturę hierarchiczną, czyli drzewo. Na górze mamy rodzica, a pod nim wiele dzieci. Tutaj Polska jest rodzicem, a Gdańsk, Kraków, Wrocław, ... dziećmi. To po której stronie umieścimy pole do przechowywania relacji zależy wyłącznie od nas. Zazwyczaj znacznie łatwiej myśli się o takiej relacji, kiedy pole umieści się po stronie dziecka (ponieważ dziecko ma tylko jednego rodzica). Czyli umieszczamy pole ForeignKey w modelu "miasto" i wpisy Gdańsk, Kraków, Wrocław posiadają referencje do wpisu Polska z modelu "państwo". 

Innym przykładem takiej relacji może być język programowania i framework. Jezyk programowania to np. Python. Framework to np. Django, Flask, Bottle. Rodzicem jest tu język programowania, dziećmi poszczególne frameworki (mówimy Python myślimy Django, Flask, Bottle... ale mówimy Django myślimy Python). Czyli pole z relacją najlepiej umieścić po stronie modelu Framework.

**Definicje modeli**

W modelach mamy dwie klasy: Language i Framework. Relacje ForignKey umieściliśmy po stronie modelu Framework (dziecko). Atrybut przechowujący relację nazwaliśmy language.

<code>class Language(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

<code>class Framework(models.Model):
    name = models.CharField(max_length=64)
    language = models.ForeignKey('Language', on_delete=models.CASCADE)
</code>
<code>
    def __str__(self):
        return f"{self.name} ({self.language})"
</code>

### C z CRUD

Operacje "C" niczym się nie różnią od tych dla pola OneToOneField. W ramach utrwalenia przypomnijmy, że mamy dwie metody.

In [4]:
from relation_app.models import Language, Framework

In [4]:
# Metad I (instancja modelu)

python = Language.objects.create(name="python")
django = Framework.objects.create(name="django", language=python)
flask = Framework.objects.create(name="flask", language=python)

In [5]:
# Metad II (id instancji modelu)

java = Language.objects.create(name="java")
spring = Framework.objects.create(name="spring", language_id=java.id)

### R z CRUD

Na początek wyświetlmy wszystkie framework-i.

In [5]:
Framework.objects.all()  # doróbmy __str__ modelom Framework i Language

<QuerySet []>

In [7]:
# A frameworki tylko dla konkretnego języka?
# Klasyczny filtr po polu language.

# 1. Pobieramy obiekt, po którym będziemy wyszukiwać.
python = Language.objects.get(name='python')

# 2. Wyszukujemy po pobranym obiekcie.
frameworks = Framework.objects.filter(language=python)
print(frameworks)

DoesNotExist: Language matching query does not exist.

In [ ]:
# wartością atrybutu reprezentującego relację jest obiekt
django = Framework.objects.get(name='django')
print(django.language)
print(type(django.language))

In [ ]:
print(dir(Language))

In [ ]:
# Realizacja relacji odwrotnej w Django
python = Language.objects.get(name='python')
python.framework_set  # powiązany menadżer obiektu


print(python.framework_set.all())
print(python.framework_set.filter(name__startswith='dj'))

In [ ]:
# Drugie zastosowanie lookupów - lookupy działają też na relacjach

# Wyciągnij wszystkie frameworki pythona

# metoda I
python = Language.objects.get(name='python')
frameworks = Framework.objects.filter(language=python)
print(frameworks)

# metoda II (z użyciem lookup)
Framework.objects.filter(language__name="python")

## ManyToMany

Ostatnia z omawianych relacji to relacja wiele-do-wielu. Dotyczy sytuacji kiedy wpisy z jednej tabeli mogą być powiązane z wielom wpisami z drugiej oraz wpisy z drugiej tabeli mogą być powiązane z wieloma wpisami z tabeli pierwszej. Przykładem takiej relacji może być tabela Aktor oraz tabela Film. Mówimy Al Pacino myślimy Scareface, Gorączka, Ojciec Chrzestny... Mówimy Ojciec Chrzestny myślimy Al Pacino, Robert DeNiro, Marlon Brando ... Relację możemy umieścić w dowolnej z powiązanych tabeli.
Jest to najbardziej złożony typ relacji, ponieważ zgodnie z zasadami normalizacji realizacja relacji wiele-do-wielu wymaga wprowadzenia tabeli pośredniej. Na szczęście django ORM o tym wszystkim wie i my nie musimy przejmować się tworzeniem nowej tabeli. Django utworzy ją za nas, a naszym zadaniem jest tylko wskazać, że chcemy mieć relację wiele-do-wielu.

**Definicje modeli**

W modelach mamy dwie klasy: Actor i Movie. Relacje ManyToMany umieszczamy po stronie Movie (ale równie dobrze moglibyśmy umieścić po stronie Actor). Atrybut przechowujący relację nazwaliśmy actors (zwróć uwagę na to, że tym razem w nazwie pola użyliśmy liczby mnogiej).

<code>class Actor(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

<code>class Movie(models.Model):
    title = models.CharField(max_length=128)
    actors = models.ManyToManyField('Actor')
</code>
<code>
    def __str__(self):
        return f"{self.title}"
</code>


Kiedy spojrzymy do bazy przekonamy się, że tabela actor nie mam kolumny movies. Zamiast tego w bazie możemy znaleźć tabelę o nazwie actor_movies (tabela pośrednia). Dodajmy kilka fimów i aktorów.

In [ ]:
from relation_app.models import Actor, Movie

movie_1 = Movie.objects.create(title='The Godfather')
movie_2 = Movie.objects.create(title='The Heat')
movie_3 = Movie.objects.create(title='The Irishman')
movie_4 = Movie.objects.create(title='Taxi Driver')
movie_5 = Movie.objects.create(title='Matrix')

actor_1 = Actor.objects.create(name='Al Pacino')
actor_2 = Actor.objects.create(name='Robert De Niro')
actor_3 = Actor.objects.create(name='Keanu Reeves')

Patrzymy do bazy i widzimy wpisy w tabeli movie i wpisy w tabeli actor, ale w tabeli movie_actors mamy 0 wpisów. W jaki sposób powiązać teraz wpisy z tabeli movie z wpisami z tabeli actor? 

In [ ]:
# Sposób I - metoda add
movie_1.actors.add(actor_1)

In [ ]:
# Sposób II - metoda add (od drugiej strony relacji)
actor_2.movie_set.add(movie_1)

In [ ]:
# Sposób III - metoda create (metoda która jednocześnie tworzy wpis i powiązuje go)
movie_1.actors.create(name="Marlon Brando")

In [ ]:
# Sposób IV - metoda set (jeżeli chcemy utworzyć kilka powiązań)
actors_list = [actor_1, actor_2]
movie_2.actors.set(actors_list)

In [ ]:
# Sposób V - metoda add też może służyc do dodania kilku powiązań jednocześnie, tylko trzeb użyć operatora rozpakowywania
movie_3.actors.add(*actors_list)

Różnica pomiędzy sposobem IV i V polega na tym, że metoda set nadpisze wszystkie istniejące powiązania movie_2 podczas gdy metoda add do istniejących powiązań movie_3 doda nowe.

In [ ]:
# Oczywiście to samo możemy robić wychodząc od drugiej strony relacji
actor_3.movie_set.add(movie_5)

Tym razem po obu stronach relacji mamych powiązane menadżery (ang. *related manager*)

In [ ]:
# Wszystki poznane wcześniej reguły pozostają w mocy

# Wszystkie filmy, w których zagrał Al Pacino - metoda I
Movie.objects.filter(actors__name="Al Pacino")

In [ ]:
# Wszystkie filmy w których zagrał Al Pacino - metoda II
actor_1.movie_set.all()

In [ ]:
# Wszyscy aktorzy, którzy zagrali w 'The Godfather'
movie_1.actors.all()